# High order statistic codes

Notebook for reading weak lensing maps maps and to run High order statistics code
All maps and codes are produced by the DESC-HOS topical team (https://github.com/LSSTDESC/HOS-Y1-prep)

- <font color='green'>Shear-2pcf (auto;cross)</font> 
- <font color='green'>map2alm </font>
- <font color='green'>Map3</font>
- integrated3PCF
- DSS
- PDF_Peaks_Minima

all modules here

In [3]:
import treecorr
#import pyccl as ccl
import numpy as np

In [4]:
#dictionary for treecorr.Catalogue input
config = {'ra_col' : 1,
          'dec_col' : 2,
          'g1_col' : 3,
          'g2_col' : 4,
          'w_col' : 5,
          'ra_units' : 'deg',
          'dec_units' : 'deg'}
#map2 nside
nside_c = 32

#Map^3 hyperparams.
# User specifications
thetas=[4,8,16,32] # Aperture scale radii in arcmin

nshells=[19,50,106] # Different shell numbers
nsides=[2048] # Different choices for nside of aperture mass maps
tomobins=[0,1,2,3,4] # Tomographic bins
seeds=[0,1,2] # Different seeds



In [ ]:
#python modules
import numpy as np
import pylab as pl
import os


In [5]:
#dependecies modules
from WLmaps import readshearsmaps
from HOScodes import hoscodes
import healsparse
import healpy as hp

In [6]:
cat = readshearsmaps(dir_name='/global/homes/j/jharno/IA-infusion/HACC-Y1/GalCat/StageIV_nz/V0/seed0/',
                     config=config)
kappacat = readkappamaps(dir_name='/global/cscratch1/sd/xuod/HOS_sims/L845/HACC150/',
                         nshells=106,
                         seeds=1,
                         nzs='kappa_zsnapshots')

In [ ]:
#for integrated3pcf TODO: read this params. as dic. or param file
filepath_map_data = './data/'
n_g = '0.6'
NSIDE = 2048

tomo_Map = 'tomo3' # will be used for computing the aperture mass
tomo_xiA = 'tomo3' # will be used for computing position dependent 2PCF
tomo_xiB = 'tomo3' # will be used for computing position dependent 2PCF

extension = 'bin_slop_0' # e.g. bin_slop_0, default_bin_slop
if ('bin_slop_0' in extension):
    bin_slop_val = 0
    extension = '_' + extension
elif ('default_bin_slop' in extension):
    extension = '_' + extension

do_2PCF = False
min_sep_tc_xi = 5 # treecorr min sep in arcmins 
max_sep_tc_xi = 170 # treecorr max sep in arcmins
nbins_tc_xi = 15 # treecorr no. of bins (should be an integer)

theta_Q_arcmins = 90 # patch radius in arcmins 
theta_T_arcmins = 90 # patch radius in arcmins 
min_sep_tc = 5 # treecorr min sep in arcmins 
max_sep_tc = 2*theta_T_arcmins-10 # treecorr max sep in arcmins
nbins_tc = 15 # treecorr no. of bins (should be an integer)

# masking fraction for patch selection
f_mask_max_Q = 0.2
f_mask_max_W = 0.2


In [ ]:
gg = treecorr.GGCorrelation(nbins = 20, min_sep=0.5, max_sep=475.5, sep_units='arcmin',bin_slop=0.01,)
# this is a treecorr object, needed for input.

for i in range(1,cat.Ntomobins+1):
    hoscodes.shear2pcf(cat.tomobins[i],gg,i) #shear2pcf
    hoscodes.map2alm(kappacat.tomobins[i],i)
    #map^3 requires an fn_out
    fn_out=dir_results+f"Map3_HACC_KappaMap_curved_nside_{nside[0]}_nshells_{nshell[0]}_tomo_{str(i)}_seed_{seed[0]}.dat"
    hoscodes.map3alm(kappacat.tomobins[i],theta=thetas[0],nside=kappacat.nshells,fn_out=fn_out)
    
    
for i in range(cat.Ntomobins):
    for j in range(cat.Ntomobins):
        hoscodes.shear2pxcf(cat.tomobins[i],gg,i,j)
        
hoscodes.integrated3PCF(tomo_map,tomo_xiA,tomo_xiB,theta_Q_arcmins,theta_T_arcmins,NSIDE,n_g,extension)
hoscodes.PDFPeaksMinima(np.concatenate(kappacat.tomobins),load=True)

In [ ]:
#hoscodes class must have all the relevant statistics to create the plots